Трекинг всех классов:

In [1]:
# import cv2
# import os
# import json
# from ultralytics import YOLO
# import numpy as np
# from pathlib import Path
# from datetime import datetime

# # Загрузка модели
# model = YOLO('best.pt')
# # model.fuse()  # Оптимизация модели для инференса

In [2]:
# def process_directory(directory_path, conf_threshold=0.5):
#     # Создаем папку results, если она не существует
#     results_dir = os.path.join(directory_path, 'results')
#     os.makedirs(results_dir, exist_ok=True)
    
#     results = []
#     for filename in os.listdir(directory_path):
#         if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
#             image_path = os.path.join(directory_path, filename)
#             result = process_image(image_path, conf_threshold)
#             if result:
#                 results.append({
#                     "filename": filename,
#                     "has_signature": "есть" if result["has_signature"] else "нет",
#                     "has_seal": "есть" if result["has_seal"] else "нет",
#                     "document_type": "",
#                     "confidence": {
#                         "signature": round(result["confidence"]["signature"], 2),
#                         "seal": round(result["confidence"]["seal"], 2)
#                     }
#                 })
    
#     # Сохраняем результаты в JSON файл
#     json_path = os.path.join(results_dir, 'analysis_results.json')
#     with open(json_path, 'w', encoding='utf-8') as f:
#         json.dump(results, f, ensure_ascii=False, indent=2)
    
#     print(f"Результаты сохранены в файл: {json_path}")
#     return results


In [1]:
import cv2
import os
import json
from ultralytics import YOLO
from pathlib import Path

# Загрузка модели
model = YOLO('best.pt')
model.fuse()  # Оптимизация модели для инференса

def process_directory(directory_path, conf_threshold=0.5):
    """Обработка всех изображений в директории и сохранение результатов в JSON"""
    # Создание директории для результатов
    results_dir = Path(directory_path) / 'results'
    os.makedirs(results_dir, exist_ok=True)
    
    # Получение списка всех изображений
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
    image_files = [f for f in os.listdir(directory_path) 
                  if os.path.isfile(os.path.join(directory_path, f)) 
                  and Path(f).suffix.lower() in image_extensions]
    
    print(f"Найдено {len(image_files)} изображений для обработки")
    
    # Список для хранения результатов
    analysis_results = []
    
    # Обработка каждого изображения
    for image_file in image_files:
        image_path = os.path.join(directory_path, image_file)
        print(f"\nОбработка {image_file}...")
        
        # Анализ изображения
        result = analyze_image(image_path, conf_threshold)
        if result:
            analysis_results.append(result)
    
    # Сохранение результатов в JSON
    json_path = results_dir / 'analysis_results.json'
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(analysis_results, f, ensure_ascii=False, indent=2)
    
    print(f"\nРезультаты сохранены в файл: {json_path}")
    return analysis_results

def analyze_image(image_path, conf_threshold=0.5):
    """Анализ одного изображения и определение наличия подписи/печати"""
    # Чтение изображения
    image = cv2.imread(image_path)
    if image is None:
        print(f"Ошибка: Не удалось прочитать изображение {image_path}")
        return None
    
    # Получение предсказаний
    results = model.predict(image, conf=conf_threshold, verbose=False)
    
    # Инициализация результатов анализа
    has_signature = False
    has_seal = False
    
    # Анализ результатов детекции
    for r in results:
        boxes = r.boxes
        for box in boxes:
            cls = int(box.cls[0])
            class_name = model.names[cls]
            
            if class_name == 'signature':
                has_signature = True
            elif class_name == 'seal':
                has_seal = True
    
    # Формируем результат
    result = {
        "filename": Path(image_path).name,
        "has_signature": "есть" if has_signature else "нет",
        "has_seal": "есть" if has_seal else "нет",
        "document_type": ""
    }
    
    return result

Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


In [2]:
# Обработка всех изображений в папке dataset_proverka
process_directory('dataset_proverka', conf_threshold=0.5)

Найдено 6 изображений для обработки

Обработка mS02noiFTahGtNZlRsUT5wRIKFWURYyQjbv7BxQN.jpeg...

Обработка ne-vydaiut-razreshenie-na-stroitelstvo-doma--361-1518619999-podpisannyj3_pdf-3.jpg...

Обработка nefras-s2-80-120-benzin-kalosha-br-2_foto_largest (1).jpg...

Обработка nefras-s2-80-120-benzin-kalosha-br-2_foto_largest.jpg...

Обработка notarialnoe-zaverenie-diploma-arabskii-alzhir.jpeg...

Обработка notarialnyi-perevod-oae-1448x2048.jpg...

Результаты сохранены в файл: dataset_proverka\results\analysis_results.json


[{'filename': 'mS02noiFTahGtNZlRsUT5wRIKFWURYyQjbv7BxQN.jpeg',
  'has_signature': 'нет',
  'has_seal': 'нет',
  'document_type': ''},
 {'filename': 'ne-vydaiut-razreshenie-na-stroitelstvo-doma--361-1518619999-podpisannyj3_pdf-3.jpg',
  'has_signature': 'есть',
  'has_seal': 'нет',
  'document_type': ''},
 {'filename': 'nefras-s2-80-120-benzin-kalosha-br-2_foto_largest (1).jpg',
  'has_signature': 'есть',
  'has_seal': 'нет',
  'document_type': ''},
 {'filename': 'nefras-s2-80-120-benzin-kalosha-br-2_foto_largest.jpg',
  'has_signature': 'есть',
  'has_seal': 'нет',
  'document_type': ''},
 {'filename': 'notarialnoe-zaverenie-diploma-arabskii-alzhir.jpeg',
  'has_signature': 'есть',
  'has_seal': 'нет',
  'document_type': ''},
 {'filename': 'notarialnyi-perevod-oae-1448x2048.jpg',
  'has_signature': 'есть',
  'has_seal': 'нет',
  'document_type': ''}]